# Archive buffer

Define the spaces and the cell factory.

In [1]:
from gym import spaces
from go_explore.inverse_model import LinearInverseModel

obs_space = spaces.Box(0, 10, (2,))
observation_space = spaces.Dict({"observation": obs_space, "goal": obs_space})
action_space = spaces.Discrete(3)
inverse_model = LinearInverseModel(obs_space.shape[0], action_size=action_space.n, latent_size=3)


Define the buffer.

In [2]:
from go_explore.archive import ArchiveBuffer

archive = ArchiveBuffer(
    buffer_size=100,
    observation_space=observation_space,
    action_space=action_space,
    inverse_model=inverse_model,
    n_envs=2,
)

Feed the buffer.

In [3]:
import numpy as np

trajectories = np.array(
    [
        [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
        [[0, 0], [1, 0], [1, 1], [1, 2], [1, 3], [0, 3], [0, 4]],
    ]
)
goals = np.array(
    [
        [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
        [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
    ]
)

observations = [
    {
        "observation": trajectories[:, i],
        "goal": goals[:, i],
    }
    for i in range(7)
]

In [4]:
for i in range(6):
    archive.add(
        obs=observations[i],
        next_obs=observations[i + 1],
        action=np.array([[0], [0]]),
        reward=np.array([0, 0]),
        done=np.ones(2) * (i == 5),
        infos=[{}, {}],
    )

In [5]:
archive.recompute_embeddings()

Try sampling trajectory method.

In [6]:
archive.sample_trajectory()

(array([[0., 1.],
        [0., 2.]], dtype=float32),
 array([[-0.29132128, -0.29377362, -0.08554482],
        [-0.17582622, -0.41293615, -0.12955344]], dtype=float32))

Here is the set of possible trajectories.

In [32]:
possible_trajectories = [
    [[0, 1]],
    [[0, 1], [0, 2]],
    [[0, 1], [0, 2], [0, 3]],
    [[0, 1], [0, 2], [0, 3], [0, 4]],
    [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5]],
    [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6]],
    [[1, 0]],
    [[1, 0], [1, 1]],
    [[1, 0], [1, 1], [1, 2]],
    [[1, 0], [1, 1], [1, 2], [1, 3]],
    [[1, 0], [1, 1], [1, 2], [1, 3], [0, 3]],
    [[1, 0], [1, 1], [1, 2], [1, 3], [0, 3], [0, 4]],
]

Check that all possible trajectories are sampled.
Also check that all sampled trajectories are possible.

In [34]:
sampled_trajectories = [archive.sample_trajectory()[0].astype(int).tolist() for _ in range(50)]  # list convinient to compare

assert np.all([trajectory in possible_trajectories for trajectory in sampled_trajectories])
assert np.all([trajectory in sampled_trajectories for trajectory in possible_trajectories])

Do the same with observation step at 2.

In [36]:
possible_trajectories = [
    [[0, 1]],
    [[0, 2]],
    [[0, 1], [0, 3]],
    [[0, 2], [0, 4]],
    [[0, 1], [0, 3], [0, 5]],
    [[0, 2], [0, 4], [0, 6]],
    [[1, 0]],
    [[1, 1]],
    [[1, 0], [1, 2]],
    [[1, 1], [1, 3]],
    [[1, 0], [1, 2], [0, 3]],
    [[1, 1], [1, 3], [0, 4]],
]
sampled_trajectories = [archive.sample_trajectory(step=2)[0].astype(int).tolist() for _ in range(50)]

assert np.all([trajectory in possible_trajectories for trajectory in sampled_trajectories])
assert np.all([trajectory in sampled_trajectories for trajectory in possible_trajectories])